In [1]:
pos_data_path = "/lustre/fsw/portfolios/llmservice/users/sghotra/data/gen/pv_methd/preproc_gsm8k_pos/full"

# read the datasets (huggingface)
# group by "ids"
# from neg dataset, remove responses w/ label=1
# merge the datasets: 50% of responses from each dataset.

In [2]:
# read the datasets (huggingface)
from datasets import load_from_disk
pos_data = load_from_disk(pos_data_path)

/lustre/fsw/portfolios/llmservice/users/sghotra/installs/miniforge3/envs/adv_verif/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
print(pos_data[0].keys())

dict_keys(['text', 'question', 'answer', 'id', 'target', 'response', 'label', 'gen_id'])


In [4]:
# group by "ids"
pos_data_grp = {}
for i in range(len(pos_data)):
    id = pos_data[i]["id"]
    
    if id in pos_data_grp:
        pos_data_grp[id].append(pos_data[i])
    else:
        pos_data_grp[id] = [pos_data[i]]


In [5]:
import random
random.seed(6730)

# merge the pos and neg datasets: 50% of responses from each dataset.

incomplete_dpoints = 0

merged_data = []
all_ids = set(pos_data_grp.keys())
for id in all_ids:
    pos_dp_h = []
    neg_dp_h = []

    if id in pos_data_grp:
        pos_dp = pos_data_grp[id]
        random.shuffle(pos_dp)
        pos_dp_h = pos_dp

        for dp in pos_dp_h:
            dp["policy"] = "pos"
    

    # if len(neg_dp_h) < 8:
    #     neg_dp_h = neg_dp.shuffle()[:8]
    #     if len(neg_dp_h) < 8:
    #         pos_dp_h = pos_dp.shuffle()[:8]
    
    # if len(pos_dp_h) < 8:
    #     pos_dp_h = pos_dp.shuffle()[:8]

    dp = pos_dp_h + neg_dp_h
    merged_data.extend(dp)

    if len(dp) < 16:
        incomplete_dpoints += 1
    
print(f"Number of incomplete datapoints: {incomplete_dpoints}")

Number of incomplete datapoints: 2607


In [6]:
print("Merged data: ", len(merged_data))
print(merged_data[0].keys())

Merged data:  115361
dict_keys(['text', 'question', 'answer', 'id', 'target', 'response', 'label', 'gen_id', 'policy'])


In [7]:
# random select 500 ids for val set
val_ids = random.sample(all_ids, 500)

train_data = []
val_data = []

for dp in merged_data:
    if dp["id"] in val_ids:
        val_data.append(dp)
    else:
        train_data.append(dp)

print("Train data: ", len(train_data))
print("Val data: ", len(val_data))


# save the datasets
from datasets import Dataset

output_dir = "/lustre/fsw/portfolios/llmservice/users/sghotra/data/gen/pv_methd/preproc_gsm8k_pos/full/splits"

train_data_dict = {key: [d[key] for d in train_data] for key in train_data[0]}
train_dataset = Dataset.from_dict(train_data_dict)

val_data_dict = {key: [d[key] for d in val_data] for key in val_data[0]}
val_dataset = Dataset.from_dict(val_data_dict)

from datasets import DatasetDict
dataset_dict = DatasetDict({
        'train': train_dataset,
        'eval': val_dataset
    })


dataset_dict.save_to_disk(output_dir)


/tmp/ipykernel_2221365/2441908778.py:2: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  val_ids = random.sample(all_ids, 500)


Train data:  107727
Val data:  7634


Saving the dataset (1/1 shards): 100%|██████████| 7634/7634 [00:00<00:00, 649584.45 examples/s]
